In [5]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
import xml.etree.cElementTree as ET
import gensim
from gensim.test.utils import common_texts

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import gensim.downloader as api
import spacy

In [2]:
path = './data/restaurants_laptop_train_with_pos.csv'
df = pd.read_csv(path)
punc = []
for p in string.punctuation:
    punc.append(p)
df["text"] = df.apply(lambda r: r["text"].lower(), axis = 1)
df = df[~df.text.isin(punc)]

In [3]:
df

,num,text,pos,aspect_tag,polarity
0,s_1,i,PRON,NAT,0
1,s_1,charge,VERB,NAT,0
2,s_1,it,PRON,NAT,0
3,s_1,at,ADP,NAT,0
4,s_1,night,NOUN,NAT,0
...,...,...,...,...,...
63226,s_3432,and,CCONJ,NAT,0
63227,s_3432,rice,NOUN,AT,0
63228,s_3432,and,CCONJ,NAT,0
63229,s_3432,glass,NOUN,AT,0


In [6]:
dataset = api.load("text8")  # load dataset as iterable
model = Word2Vec(dataset)
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

In [38]:
train, test = train_test_split(df, test_size = 0.2, random_state=42)
le = preprocessing.LabelEncoder()

y_test= test.aspect_tag.values
y_test = le.fit_transform(y_test)
y_train = train.aspect_tag.values
y_train = le.fit_transform(y_train)
train["word2vec"] = train.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1) 
test["word2vec"] = test.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)

C:\Users\sr_te\AppData\Local\Temp/ipykernel_13716/2943963045.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["word2vec"] = train.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)
C:\Users\sr_te\AppData\Local\Temp/ipykernel_13716/2943963045.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["word2vec"] = test.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)


In [5]:
train

,num,text,pos,aspect_tag,polarity,word2vec
5766,s_335,good,ADJ,NAT,0,"[-1.9445202, 0.71691823, -0.4608151, 0.2600984..."
14607,s_843,thai,PROPN,AT,1,"[0.20540255, 0.96596104, -0.3000787, -0.273339..."
8342,s_479,truly,ADV,NAT,0,"[-1.6709162, -0.10299236, 0.39241987, -1.02344..."
1462,s_86,when,SCONJ,NAT,0,"[1.3618017, 0.528671, 0.18292685, 0.3734613, -..."
6547,s_383,ranging,VERB,NAT,0,"[0.6964434, -0.98072594, -1.1850027, -0.045366..."
...,...,...,...,...,...,...
33705,s_1938,the,DET,NAT,0,"[-0.32336417, -0.08010307, 0.35241595, 0.03340..."
6093,s_355,thoroughly,ADV,NAT,0,"[-0.38939166, 0.2582531, 0.28199467, -0.551408..."
971,s_59,almost,ADV,NAT,0,"[0.0021587578, -1.9058623, 1.2874553, -3.16862..."
17888,s_1033,30,NUM,NAT,0,"[-0.14496428, -0.33685055, -0.15259372, -1.473..."


In [63]:
X_train = train.word2vec[1000]
X_test = test.word2vec.to_list()
X_train

array([-1.29001275e-01,  5.46103716e-02,  3.17520909e-02, -3.62295955e-02,
       -1.50131464e-01, -4.49253380e-01,  9.68820825e-02,  5.46176076e-01,
        1.71248898e-01,  3.24651599e-01,  1.88228220e-01, -2.82642633e-01,
       -6.21698871e-02, -1.44591197e-01, -1.41891465e-01, -5.33352010e-02,
        2.74052233e-01, -2.08114833e-01, -2.13746160e-01, -2.14221776e-01,
        2.51619369e-01,  1.75015286e-01, -1.24911271e-01,  4.32970788e-04,
       -1.91607460e-01,  1.49356052e-01, -6.00321516e-02, -3.08057845e-01,
       -1.54705063e-01,  4.67313737e-01,  2.61764258e-01,  2.49305880e-03,
       -4.86798994e-02, -4.66249846e-02, -3.96305919e-02,  1.90602928e-01,
       -2.61783361e-01, -6.25234693e-02, -1.69621944e-01,  1.44394517e-01,
        4.66490872e-02, -2.59971678e-01, -2.06387550e-01, -5.21993637e-01,
        3.98182385e-02,  4.10293788e-03, -3.77813935e-01,  1.14760719e-01,
       -1.51055127e-01,  2.54739583e-01, -2.65932411e-01,  9.67392419e-03,
       -5.21118902e-02, -

In [53]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [41]:
pred = lr.predict(X_test)
pred, y_test

(array([1, 1, 1, ..., 1, 1, 1]), array([1, 1, 1, ..., 1, 1, 1]))

In [42]:
print('Testing accuracy: {:.2f}%'.format(lr.score(X_test, y_test)*100))

Testing accuracy: 90.47%
